<span style="float:left;">Licence CC BY-NC-ND</span><span style="float:right;">Thierry Parmentelat &amp; Arnaud Legout&nbsp;<img src="media/both-logos-small-alpha.png" style="display:inline"></span><br/>

# coroutines et awaitables

# protocole awaitable

| instruction | classe d'objets  | protocole | exemple | 
|-------------|------------------|-----------|---------|
|   `for`     |   itérables      |  `__iter__` | liste, ensemble |
|   `with`    | context managers | `__enter__` & `__exit__` | fichier |
|   `await`   |   awaitables     | `__await__` | objet coroutine |

# `__await__` renvoie un itérateur

In [12]:
class Awaitable():
    def __await__(self):
        print("awaiting")
        yield 10

In [13]:
# il nous faut au moins une coroutine
# pour pouvoir faire await
async def main():
    await Awaitable()

In [14]:
# l'objet coroutine
coro = main()

In [15]:
coro.send(None)

awaiting


10

# un peu moins simple 

In [17]:
class Awaitable2():
    def __await__(self):
        print("step1")
        yield 10
        print("step2")
        yield 20
        return "FINAL"

In [20]:
# boilerplate
async def main():
    return await Awaitable2()

In [21]:
# l'objet coroutine
coro = main()


/opt/local/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning: coroutine 'main' was never awaited
  


In [22]:
coro.send(None)

step1


10

In [23]:
coro.send(None)

step2


20

In [24]:
try:
    coro.send(None)
except Exception as e:
    x = e
    print('OOPS', type(e), e.value)

OOPS <class 'StopIteration'> FINAL


# Plusieurs travaux en même temps

In [25]:
coro1 = main()

In [26]:
coro2 = main()

In [27]:
coro1.send(None)

step1


10

In [28]:
coro2.send(None)

step1


10

In [29]:
coro1.send(None)

step2


20

In [30]:
coro2.send(None)

step2


20

In [31]:
try:
    coro1.send(None)
except Exception as e:
    x = e
    print('OOPS', type(e), e.value)

OOPS <class 'StopIteration'> FINAL


In [32]:
try:
    coro2.send(None)
except Exception as e:
    x = e
    print('OOPS', type(e), e.value)

OOPS <class 'StopIteration'> FINAL


# pile, await et yield

In [33]:
class AwaitableValue:
    counter = 0
    def __init__(self, value):
        self.value = value
    def __await__(self):
        AwaitableValue.counter += 1
        # redonner la main à la boucle
        yield f"y {self.counter}"
        # retourné à await
        return self.value

In [34]:
async def w1():
    return await AwaitableValue(1)

async def w2():
    return await w1() + await w1()

async def w3():
    return await w2() + 1

async def w4():
    return await w3() + 1

coro = w4()

In [36]:
coro.send(None)

'y 2'

In [ ]:
coro.send(None)

In [37]:
try:
    coro.send(None)
except Exception as e:
    x = e
    print('OOPS', type(e), e.value)

OOPS <class 'StopIteration'> 4


# [animation](single-stack/index.html)

# Conclusion

### protocole itérable

### méthode `send()`

### liée aux `yield` 